In [44]:
from flask import Flask, request, jsonify, render_template
from heap import MinHeap
import copy

app = Flask(__name__)

class PrefixTreeNode:
    def __init__(self,string = None, json = None, frequency = 1):
        self.json = json
        self.string = string
        self.frequency = frequency
    def __lt__(self, other):
        if self.frequency < other.frequency:
            return True
        return False        
    def __gt__(self, other):
        if self.frequency > other.frequency:
            return True
        return False 
    def __le__(self, other):
        if self.frequency <= other.frequency:
            return True
        return False        
    def __ge__(self, other):
        if self.frequency >= other.frequency:
            return True
        return False
class PrefixTree:
    #TODO реализация класса prefix tree, методы как на лекции + метод дать топ 10 продолжений. Скажем на строку кросс выдаем кроссовки, кроссовочки итп. Как хранить топ? 
    #Решать вам. Можно, конечно, обходить все ноды, но это долго. Дешевле чуток проиграть по памяти, зато отдавать быстро (скажем можно взять кучу)
    #В терминальных (конечных) нодах может лежать json с топ актерами.
    def __init__(self):
        self.root = [{}]
        
    #def __get__(self, key):
    #    return self.children[key]
        
    def add(self, string = None, json = None, frequency = 1):
        if self.check(string):
            return
        wrk_dict = self.root
        
        for i in string:
            if i in wrk_dict[0]:
                wrk_dict = wrk_dict[0][i]
            else:
                wrk_dict[0][i] = [{}, MinHeap([])]
                wrk_dict = wrk_dict[0][i]
            node = PrefixTreeNode(string, json, frequency)
            if len(wrk_dict[1].array) < 10:
                wrk_dict[1].add(node)
            else:
                min_node = wrk_dict[1].extract_minimum()
                if node.frequency > min_node.frequency:
                    wrk_dict[1].add(node)
                else:
                    wrk_dict[1].add(min_node)
        wrk_dict.append(json)        
        
    def check(self, string):        
        wrk_dict = self.root
        for i in string:
            if i in wrk_dict[0]:
                wrk_dict = wrk_dict[0][i]
            else:
                return False
        if len(wrk_dict) == 3:
            return True
        return False
    
    def check_part(self, string):
        wrk_dict = self.root
        for i in string:
            if i in wrk_dict[0]:
                wrk_dict = wrk_dict[0][i]
            else:
                return False
        return True
    
    def top10(self, string):
        top = []
        if self.check_part(string):
            wrk_dict = self.root
            for i in string:
                wrk_dict = wrk_dict[0][i]
            node_array = copy(wrk_dict[1].array)
            node_array.sort(reverse = True)
            for i in node_array:
                sudgest = {'sudgest': s.string, 'json': s.json}
                top.append(sudgest)
        return top
        

def init_prefix_tree(filename):
    #TODO в данном методе загружаем данные из файла. Предположим вормат файла "Строка, чтобы положить в дерево" \t "json значение для ноды" \t частота встречаемости
    f = open(filename, 'r')
    for line in f:
        data = line.split('\t')
        PrefixTree.add(data[0], data[1], data[2])
    f.close()




In [34]:
import pandas as pd
data = pd.read_pickle('parser_data.pkl')
df_pkl = pd.DataFrame(data)
title = []
title = df_pkl.Title

In [58]:
pTree = PrefixTree()
for i in title:
    pTree.add(i)

In [54]:
@app.route("/get_sudgest/<string>", methods=['GET', 'POST'])
def return_sudgest(string):
    #TODO по запросу string вернуть json, c топ-10 саджестами, и значениями из нод
    data = pTree.top10(string)
    return jsonify(data)
    

@app.route("/")
def hello():
    #TODO должна возвращатьс инструкция по работе с сервером
    return render_template('hello.html')

if __name__ == "__main__":
    app.run()

AssertionError: View function mapping is overwriting an existing endpoint function: return_sudgest